In [1]:
import os
import shutil
import pandas as pd
from tqdm import tqdm      
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
import time
from datetime import datetime

### 나우리 나스 파일 옮기기

##### depth2,3 제거하고 파일명에 채무자키 없는 거 추가하기

In [7]:
# 문서구분 폴더를 기준으로 채무자키 파일명에 넣어서 파일 저장하기
path = r'D:\nas다운로드\나우리서버\원인서류'
path_dst = r'D:\nas다운로드\upload\원인서류'
path_s = r'C:\Users\SL\Desktop\test\log\success'
path_f = r'C:\Users\SL\Desktop\test\log\fail'
path_d = r'C:\Users\SL\Desktop\test\log\dir'

file_list = os.walk(path)
p_key = re.compile(r"[\d]{8}(?!\d)") #match


result_f = []
result_d = []
error = []
os.chdir(r'D:\\')

In [ ]:
for root, dirs, files in tqdm(file_list) :
    # root : 삭제할 채무자키 폴더
    #os.chdir(root)
    for f in files :
        try : 
            n = f
            if not p_key.match(n) : # 파일명에 채무자키 없는 경우
                key = root.split(os.path.sep)[-1]
                # == key = os.path.split(root)[1]
                # print("key : ", key)
                n = key + " " + n
                # print("n : ", n)
            else : pass

            result_f.append(ff.re_name(join(root, f), join(path_dst, n)))
        except Exception as e:
            print("===================================")
            print(datetime.today().strftime("%H:%M:%S"))
            print(f, e.__class__, e, sep=" : ")
            print(traceback.format_exc())
            error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아
    #os.chdir(r'D:\nas다운로드\나우리서버')

# for root, dirs, files in tqdm(os.walk(path)) :
#     for dir in dirs :
#         try : 
#             os.removedirs(dir)
#             result_d.append([dir]) #log는 리스트를 인자로 받는다.
#         except Exception as e:
#             print("===================================")
#             print(datetime.today().strftime("%H:%M:%S"))
#             print(root, e.__class__, e, sep=" : ")
#             print(traceback.format_exc())
#             error.append([dir, e.__class__, e])
#             continue # 반복문 계속 돌아


ff.write_log_csv(result_f, path_s)
ff.write_log_csv(error, path_f)
#ff.write_log_csv(result_d, path_d)


# 남아있는 디렉토리 확인
# remain = []
# for d in tqdm(result_d, total=len(result_d)) :
#     if os.path.isdir(d) :
#         remain.append(d)
# if len(remain) >0 : print(len(remain))
# else : print("모든 디렉토리 삭제 완료")

##### 파일명에 문서구분 없는 거 별도 폴더로 옮기기

In [28]:
path = r'D:\nas다운로드\upload\원인서류'
nodocuPath = r'D:\nas다운로드\upload\원인서류\nodocu' ####################만들어
##########################################
standard = "원인서류"
p1=re.compile(r"원인\s?서류|(입회|가입|카드)\s?신청서|(신용)?\s?대출\s?(신청서)?|(분할)?\s?약정서|녹취록?|통화\s?(내용|내역)|(대출)?\s?원장|마이너스\s?대출")
##########################################

p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif)$', re.I)
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total = len(file_list)

cnt = 0
for f in tqdm(file_list, total=total) :
    if p1.search(f) :
        n = p1.sub(standard, f)
        if f != n :
            os.rename(join(path, f), join(path, n))
    else :
        os.rename(join(path, f), join(nodocuPath, f))
        cnt += 1
print(f'{total}개 중 {cnt}개 nodou로 이동')


100%|██████████| 3518/3518 [00:00<00:00, 12618.52it/s]

3518개 중 0개 nodou로 이동


In [ ]:
##### 끝에 문서구분 붙이고 중간에 있는 일련번호 지우자

### 잘못 만든 상위 폴더 지우기

##### 부산 > nas

In [60]:
df = pd.read_excel(r"C:\Users\SL\Desktop\통합 문서1.xlsx", names=['src_dir', 'src_f', 'dst_f', 'dst_dir'], header=None)
total = len(df.index)
print(total)
path_s = r'C:\Users\SL\Desktop\test\log\success'
path_f = r'C:\Users\SL\Desktop\test\log\fail'
path_d = r'C:\Users\SL\Desktop\test\log\dir'

result_f = []
result_d = []
error = []

#temp_cwd = os.getcwd()
for key, row in tqdm(df.iterrows(), total=total):
    dst_dir = row.dst_dir
    dst_f = row.dst_f
    miss_dir = (join(dst_dir, dst_f)) #파일명들어간 폴더
    
    # print("miss_dir : ",miss_dir)
    # print("dst_dir : ", dst_dir)

    for f in os.listdir(miss_dir) :
        os.chdir(miss_dir)
        # print("f : ",f)
        if os.path.isfile(f) :
            try : 
                result_f.append(ff.re_name(f, join(dst_dir, f)))
                
            except Exception as e:
                print("===================================")
                print(datetime.today().strftime("%H:%M:%S"))
                print(f, e.__class__, e, sep=" : ")
                print(traceback.format_exc())
                error.append([f, e.__class__, e])
                continue # 반복문 계속 돌아
    os.chdir(dst_dir)
    try : 
        os.rmdir(miss_dir)
        result_d.append(miss_dir)
    except Exception as e:
        print("===================================")
        print(datetime.today().strftime("%H:%M:%S"))
        print(f, e.__class__, e, sep=" : ")
        print(traceback.format_exc())
        error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아

ff.write_log_csv(result_f, path_s)
ff.write_log_csv(result_d, path_d)
ff.write_log_csv(error, path_f)

# 남아있는 디렉토리 확인
remain = []
for d in tqdm(result_d, total=len(result_d)) :
    if os.path.isdir(d) :
        remain.append(d)
if len(remain) >0 : print(remain)
else : print("모든 디렉토리 삭제 완료")


2671


##### 구나스 등초본

In [19]:
path_s = r'C:\Users\SL\Desktop\test\log\success'
path_f = r'C:\Users\SL\Desktop\test\log\fail'
path_d = r'C:\Users\SL\Desktop\test\log\dir'
path = r'D:\nas다운로드\upload\등초본' #폴더만 있음

result_f = []
result_d = []
error = []

file_list = os.listdir(path)
total = len(file_list)
#temp_cwd = os.getcwd()
for dir in tqdm(file_list, total=total):
    
    miss_dir = join(path, dir) #파일명들어간 폴더
    
    # print("miss_dir : ",miss_dir)
    # print("dst_dir : ", dst_dir, type(dst_dir))
    # print("dst_f : ", dst_f)

    os.chdir(miss_dir)
    for f in os.listdir(miss_dir) :
        # print("f : ",f)
        if os.path.isfile(f) :
            try : 
                result_f.append(ff.re_name(f, join(path, dir)))
                
            except Exception as e:
                print("===================================")
                print(datetime.today().strftime("%H:%M:%S"))
                print(f, e.__class__, e, sep=" : ")
                print(traceback.format_exc())
                error.append([f, e.__class__, e])
                continue # 반복문 계속 돌아
    os.chdir(path)
    try : 
        os.rmdir(miss_dir)
        result_d.append(miss_dir)
    except Exception as e:
        print("===================================")
        print(datetime.today().strftime("%H:%M:%S"))
        print(f, e.__class__, e, sep=" : ")
        print(traceback.format_exc())
        error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아

ff.write_log_csv(result_f, path_s)
ff.write_log_csv(result_d, path_d)
ff.write_log_csv(error, path_f)

# 남아있는 디렉토리 확인
remain = []
for d in tqdm(result_d, total=len(result_d)) :
    if os.path.isdir(d) :
        remain.append(d)
if len(remain) >0 : print(remain)
else : print("모든 디렉토리 삭제 완료")


100%|██████████| 2806/2806 [00:02<00:00, 1129.32it/s]


log 파일이 비어있습니다. path : C:\Users\SL\Desktop\test\log\fail


100%|██████████| 2806/2806 [00:00<00:00, 23505.62it/s]

모든 디렉토리 삭제 완료


In [40]:
os.getcwd()

'c:\\'

In [34]:
os.chdir("c://")